In [1]:
import pandas as pd
import altair as alt
import numpy as np
import os
import sys

source_dir = '/Users/matthewdeverna/Documents/Projects/mean_field_quals/src'
sys.path.append(source_dir)


from simulations import run_simulation, get_peak_day

### Varying $\beta$ with ordinary proportion = 1

In [125]:
beta_values = np.arange(.01, 1.01, .01)

less = [0.2, 0.4, 0.6, 0.8, 1. ]

totals = []
r0s_ = []
peak_days = []

infection_flows = dict()

for beta in beta_values:
    S_o, S_m, I_o, I_m, R_o, R_m, r0s = run_simulation(
        frac_ord= 1,
        prop_infec=.001,
        num_days=100,
        beta_ord=beta,
        recovery_days=5,
        beta_mult=2
    )
    total_ord_inf = max(R_o)
    
    if np.round(beta,2) in less:

        infection_flows[beta] = I_o
    
    r0s_.append( (r0s[0], beta) )
    
    totals.append((beta, total_ord_inf))

In [126]:
total_frame = pd.DataFrame(totals, columns = ['beta','total_infected'])

In [127]:
total_frame

,beta,total_infected
0,0.01,0.001053
1,0.02,0.001111
2,0.03,0.001176
3,0.04,0.001250
4,0.05,0.001333
...,...,...
95,0.96,0.997056
96,0.97,0.997307
97,0.98,0.997539
98,0.99,0.997753


In [128]:
r0frames = pd.DataFrame(r0s_, columns=['r0', 'beta'])

In [129]:
beta_val = np.round(r0frames[r0frames['r0'] > 1]['beta'].min(), 2)

rule_text_df = pd.DataFrame({
    'x' : [r0frames[r0frames['r0'] > 1]['beta'].min()],
    'y1' : [0],
    'y2' : [3],
    'text' : [f"R0 > 1 (beta = {beta_val})"]
})

r0fig1 = alt.Chart(r0frames).mark_point(color='black', size=50).encode(
    x = alt.X(
        'beta:Q',
        axis = alt.Axis(
            title = None,
            labelExpr="(datum.value*100) % 10 ? null : datum.label"
        )
    ),
    y = alt.Y(
        "r0:Q",
        title = "R0"
    )
).properties(width=600, height=200) + alt.Chart(rule_text_df).mark_rule(color = 'red').encode(
    x = 'x:Q',
    y = 'y1:Q',
    y2 = 'y2:Q'
) + alt.Chart(rule_text_df).mark_text(
    dx=-20,
    dy=-10,
    fontSize=13
).encode(
    x = 'x:Q',
    y = 'y2:Q',
    text = 'text'
)


tots_fig = alt.Chart(total_frame).mark_point(color='black', size=50).encode(
    x = alt.X(
        "beta:Q",
        axis = alt.Axis(
            title=['beta','(no misinformed population)'],
            labelExpr="(datum.value*100) % 10 ? null : datum.label"
        )
    ),
    y = alt.Y("total_infected:Q", title='Proportion of population infected')
).properties(width=600)



combined_fig = alt.vconcat(r0fig1, tots_fig)

combined_fig.configure_axis(
    titleFontSize=15,
    labelFontSize=14,
    labelAngle=0
).configure_legend(
    titleFontSize=15,
    labelFontSize=14,
)

alt.VConcatChart(...)

In [130]:
infection_flows.keys()

dict_keys([0.2, 0.4, 0.6, 0.8, 1.0])

In [131]:
infections = pd.DataFrame(infection_flows).reset_index()
infections = infections.rename(columns={'index':'day'})

infections.melt(id_vars='day')

,day,variable,value
0,0,0.2,1.000000e-03
1,1,0.2,9.998000e-04
2,2,0.2,9.995601e-04
3,3,0.2,9.992803e-04
4,4,0.2,9.989607e-04
...,...,...,...
495,95,1.0,1.064079e-08
496,96,1.0,8.534438e-09
497,97,1.0,6.845038e-09
498,98,1.0,5.490057e-09


In [133]:
info = []

for beta, infects in infection_flows.items():
    peak_x = get_peak_day(infects)
    peak_y = max(infects)
    text = f"{np.round(peak_y,2)}"
    info.append( (peak_x, peak_y, text) )

In [146]:
text_dfs = pd.DataFrame(info, columns = ['peak_x', 'peak_y', 'text'])
text_dfs

,peak_x,peak_y,text
0,0,0.001000,0.0
1,36,0.161499,0.16
2,22,0.324923,0.32
3,16,0.447510,0.45
4,13,0.541865,0.54


In [147]:
text_dfs = text_dfs[text_dfs['peak_x']>0]


In [150]:
lines = alt.Chart(infections.melt(id_vars='day')).mark_line(size=2, color = 'black').encode(
    x = alt.X(
        'day:Q',
        title = ["Time", "(days)"]
    ),
    y = alt.Y(
        'value:Q',
        title = ["Proportion of population infected", "(no misinformed group)"],
        scale = alt.Scale(domain=(0,1))
    ),
    strokeDash = alt.StrokeDash(
        'variable:N',
        title = "beta"
    )
).properties(width=600)

texts = alt.Chart(text_dfs).mark_text(dy=-7,dx=10,fontSize=14).encode(
    x = 'peak_x:Q',
    y = 'peak_y:Q',
    text = 'text'
)

alt.layer(lines,texts).configure_axis(
    titleFontSize=15,
    labelFontSize=14,
    labelAngle=0
).configure_legend(
    titleFontSize=15,
    labelFontSize=14,
)

alt.LayerChart(...)

### Varying $\beta$ with half misinformed and half ordinary

In [24]:
beta_values = np.arange(.01, 1.01, .01)

totals = []
r0s_ = []
peak_days = []

infection_flows = dict()

for beta in beta_values:
    S_o, S_m, I_o, I_m, R_o, R_m, r0s = run_simulation(
        frac_ord= .5,
        prop_infec=.001,
        num_days=100,
        beta_ord=beta,
        recovery_days=5,
        beta_mult=2
    )
    total_ord_inf = max(R_o)
    total_mis_inf = max(R_m)
    
    if beta in np.arange(.2, 1.2, .2):
        comb_inf = np.array(I_o) + np.array(I_m)

        infection_flows[beta] = comb_inf
    
    peak_day_o = get_peak_day(I_o)
    peak_day_m = get_peak_day(I_m)
    peak_day_comb = get_peak_day(np.array(I_o) + np.array(I_m))
    peak_days.append( (peak_day_o, peak_day_m, peak_day_comb, beta) )
    
    r0s_.append(list(r0s)+[beta])
    
    totals.append((total_ord_inf, total_mis_inf, beta))

In [25]:
r_naughts = pd.DataFrame(r0s_, columns = ['ord_r0', 'mis_r0', 'weighted_avg_r0', 'beta'])

In [26]:
melted_r0 = r_naughts.melt(id_vars='beta')
melted_r0.variable.unique()

mapper = {
    'ord_r0' : 'ordinary',
    'mis_r0' : 'misinformed',
    'weighted_avg_r0' : 'all'
}

melted_r0.variable = melted_r0.variable.map(mapper)

In [27]:
totals_frame = pd.DataFrame(totals, columns = ['total_ord_inf', 'total_mis_inf', 'beta'])
totals_frame['total'] = totals_frame['total_ord_inf'] + totals_frame['total_mis_inf']

totals_frame

,total_ord_inf,total_mis_inf,beta,total
0,0.000027,0.001054,0.01,0.001081
1,0.000059,0.001117,0.02,0.001176
2,0.000097,0.001193,0.03,0.001290
3,0.000143,0.001285,0.04,0.001428
4,0.000200,0.001399,0.05,0.001598
...,...,...,...,...
95,0.498561,0.498949,0.96,0.997509
96,0.498676,0.498956,0.97,0.997632
97,0.498783,0.498963,0.98,0.997746
98,0.498882,0.498970,0.99,0.997852


In [28]:
melted_totals = totals_frame.melt(id_vars = 'beta')

mapper = {
    "total_ord_inf" : "ordinary",
    "total_mis_inf" : "misinformed",
    "total" : "all",
}

melted_totals.variable = melted_totals.variable.map(mapper)

melted_totals.beta = np.round(melted_totals.beta, 2)
melted_totals


,beta,variable,value
0,0.01,ordinary,0.000027
1,0.02,ordinary,0.000059
2,0.03,ordinary,0.000097
3,0.04,ordinary,0.000143
4,0.05,ordinary,0.000200
...,...,...,...
295,0.96,all,0.997509
296,0.97,all,0.997632
297,0.98,all,0.997746
298,0.99,all,0.997852


In [37]:
beta_val = np.round(r_naughts[r_naughts['weighted_avg_r0'] > 1]['beta'].min(), 2)

rule_text_df = pd.DataFrame({
    'x' : [r_naughts[r_naughts['weighted_avg_r0'] > 1]['beta'].min()],
    'y1' : [0],
    'y2' : [3],
    'text' : [f"R0 > 1 (beta = {beta_val})"]
})

r0_fig = alt.Chart(melted_r0).mark_point(color='black', size = 50).encode(
    x = alt.X(
        'beta:Q',
        axis = alt.Axis(
            title=None,
            labelExpr="(datum.value*100) % 10 ? null : datum.label"
        )
    ),
    y = alt.Y("value:Q", title = "R0"),
    shape = alt.Y("variable:N")
).properties(width=600, height=200) + alt.Chart(rule_text_df).mark_rule(color = 'red').encode(
    x = 'x:Q',
    y = 'y1:Q',
    y2 = 'y2:Q'
) + alt.Chart(rule_text_df).mark_text(
    dx=-20,
    dy=-10,
    fontSize=13
).encode(
    x = 'x:Q',
    y = 'y2:Q',
    text = 'text'
)


In [38]:
beta_fig = alt.Chart(melted_totals).mark_point(color='black', size=50).encode(
    x = alt.X(
        "beta:Q",
        axis = alt.Axis(
            title = ['beta_o', '(beta_m = 2*beta_o)'],
            labelExpr="(datum.value*100) % 10 ? null : datum.label"
        ),
#         scale = alt.Scale(range=(0,1))
    ),
    y = alt.Y("value:Q", title='Total proportion infected'),
    shape = alt.Shape("variable:N", title='population')
).properties(width=600)


both = alt.vconcat(r0_fig, beta_fig)

both.configure_axis(
    titleFontSize=15,
    labelFontSize=14,
    labelAngle=0
).configure_legend(
    titleFontSize=15,
    labelFontSize=14,
)

alt.VConcatChart(...)

In [11]:
peak_frame = pd.DataFrame(peak_days, columns=['ordinary','misinformed', 'combined','beta'])

In [12]:
peak_frame

,ordinary,misinformed,combined,beta
0,5,0,0,0.01
1,5,0,0,0.02
2,5,0,0,0.03
3,5,0,0,0.04
4,6,0,0,0.05
...,...,...,...,...
95,13,13,13,0.96
96,13,13,13,0.97
97,13,13,13,0.98
98,13,13,13,0.99


In [17]:
peak_frame.melt(id_vars='beta').head(50)

,beta,variable,value
0,0.01,ordinary,5
1,0.02,ordinary,5
2,0.03,ordinary,5
3,0.04,ordinary,5
4,0.05,ordinary,6
5,0.06,ordinary,6
6,0.07,ordinary,7
7,0.08,ordinary,7
8,0.09,ordinary,8
9,0.10,ordinary,9


In [18]:
alt.Chart(peak_frame.melt(id_vars='beta')).mark_point().encode(
    x = 'beta:Q',
    y = 'value:Q',
    shape = 'variable:N'
).properties(width=600)

alt.Chart(...)

In [43]:

infections = pd.DataFrame(infection_flows).reset_index()

In [40]:
infections['variable'].unique()

array(['index', 0.2, 0.4, 0.8, 1.0], dtype=object)

In [45]:
infections = infections.rename(columns={'index':'day'})

In [47]:
infections.melt(id_vars='day')

,day,variable,value
0,0,0.2,1.000000e-03
1,1,0.2,1.099600e-03
2,2,0.2,1.209010e-03
3,3,0.2,1.329174e-03
4,4,0.2,1.461121e-03
...,...,...,...
395,95,1.0,1.064079e-08
396,96,1.0,8.534438e-09
397,97,1.0,6.845038e-09
398,98,1.0,5.490057e-09


In [48]:
alt.Chart(infections.melt(id_vars='day')).mark_line().encode(
    x = 'day:Q',
    y = 'value:Q',
    color = 'variable:N'
)

alt.Chart(...)

In [38]:
infections

,index,variable,value
0,0,0.2,1.000000e-03
1,1,0.2,1.099600e-03
2,2,0.2,1.209010e-03
3,3,0.2,1.329174e-03
4,4,0.2,1.461121e-03
...,...,...,...
395,395,1.0,1.064079e-08
396,396,1.0,8.534438e-09
397,397,1.0,6.845038e-09
398,398,1.0,5.490057e-09
